In [1]:
import pandas as pd
import numpy as np
# 读取CSV文件


import argparse

from tqdm import tqdm, trange
# tqdm函数用于为循环或迭代器创建进度条。
# 它可以用于显示需要很长时间才能完成的任务的进度，例如数据处理或模型训练。
# trange函数类似于Python中的range函数，但它还创建了有指定迭代次数的进度条。
# 这允许您实时查看循环的进度，因此可以更好地跟踪任务执行情况。




In [2]:
# import my_model   # my_model是自己写的一个文件
# import data # data是一个自己写的文件
import torch.nn as nn
"""
这里导入了pytorch深度学习的nn模块，这个模块提供了神经网络层、损失函数
和优化器等工具的类，通过这个模块可以方便地构建和训练神经网络模型。
"""
min_loss = float('inf')

In [3]:
criterion = nn.MSELoss()  # 使用均方误差损失函数计算MSE


In [4]:
import math

import torch
import numpy as np
import  pickle
import os
import json

In [5]:
class DataIterator(object):
    def __init__(self, x_data,x_mask_data,x_edge_data, args):
        self.x_data,self.x_mask_data,self.x_edge_data,=x_data,x_mask_data,x_edge_data,
        #date跟fearture的分开
        # 虽然x_date是一个二维数组，但是二维数组中的每个元素都是一个列表，每个列表的内容是一个日期id和三十五个F特征还有要预测的两个目标值
        # x_data[:,:,0]取出来的是日期
        # x_data[:,:,1:-2]取出来的是三十五个特征
        # x_data[:,:,-2:]取出来的是两个预测值也就是
        self.x_date,self.x_feature,self.x_tags=self.x_data[:,:,0],self.x_data[:,:,1:-2],x_data[:,:,-2:]
        # print(self.x_date.shape,self.x_feature.shape,self.x_tags.shape)
        self.args = args
        #通过数据总数除掉每个批次的数据数目args.batch_size来算出一共多少个批次
        self.batch_count = math.ceil(len(x_data)/args.batch_size)

        #get_batch函数是用来获取某个训练批次的数据的，index代表批次号
    def get_batch(self, index):
        x_date = []
        x_feature = []
        x_mask_data=[]
        x_edge_data = []
        x_tags = []


        for i in range(index * self.args.batch_size,
                       min((index + 1) * self.args.batch_size, len(self.x_data))):

            x_date.append(self.x_date[i])
            x_feature.append(self.x_feature[i].float() )

            # print(self.x_mask_data[i].shape)
            x_mask_data.append(self.x_mask_data[i])
            # print(self.x_edge_data[i].shape)
            x_edge_data.append(self.x_edge_data[i])
            x_tags.append(self.x_tags[i].float() )

        x_date = torch.stack(x_date).to(self.args.device)
        # x_feature=torch.DoubleTensor(torch.stack(x_feature)).to(self.args.device)
        x_feature = torch.FloatTensor(torch.stack(x_feature)).to(self.args.device)
        x_mask_data = torch.stack(x_mask_data).to(self.args.device)
        x_edge_data = torch.stack(x_edge_data).to(self.args.device)
        x_tags = torch.stack(x_tags).to(self.args.device)


        return  x_date,x_feature,x_mask_data,x_edge_data,x_tags

In [6]:
class GraphConvolution(nn.Module):
    def __init__(self, input_dim, output_dim, dropout, bias=False):
        super(GraphConvolution, self).__init__()
        self.dropout = nn.Dropout(dropout)
        self.weight = nn.Parameter(torch.Tensor(input_dim, output_dim))
        nn.init.xavier_uniform_(self.weight)  # xavier初始化，就是论文里的glorot初始化
        if bias:
            self.bias = nn.Parameter(torch.Tensor(output_dim))
            nn.init.zeros_(self.bias)
        else:
            self.register_parameter('bias', None)

    def forward(self, inputs, adj):
        # inputs: (N, n_channels), adj: sparse_matrix (N, N)
        support = torch.mm(self.dropout(inputs), self.weight)
        output = torch.spmm(adj, support)
        if self.bias is not None:
            return output + self.bias
        else:
            return output

In [7]:
class GCN(nn.Module):
    def __init__(self, n_features, hidden_dim, dropout, n_classes):
        super(GCN, self).__init__()
        self.gc1 = GraphConvolution(n_features, hidden_dim, dropout)
        self.gc2 = GraphConvolution(hidden_dim, n_classes, dropout)
        self.relu = nn.ReLU()

    def forward(self, inputs, adj):
        x = inputs
        x = self.relu(self.gc1(x, adj))
        x = self.gc2(x, adj)
        return x

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F


In [9]:
class GraphAttentionLayer(nn.Module):
    def __init__(self, in_features, out_features, dropout, alpha, concat):
        super(GraphAttentionLayer, self).__init__()
        self.dropout = dropout
        self.in_features = in_features
        self.out_features = out_features
        self.alpha = alpha
        self.concat = concat

        self.W = nn.Parameter(torch.zeros(in_features, out_features))
        self.a = nn.Parameter(torch.zeros(2 * out_features, 1))
        nn.init.xavier_uniform_(self.W.data, gain=1.414)
        nn.init.xavier_uniform_(self.a.data, gain=1.414)

        self.leakyrelu = nn.LeakyReLU(self.alpha)

    def forward(self, h, adj):
        '''
        h: (N, in_features)
        adj: sparse matrix with shape (N, N)
        p
        '''
        adj=torch.squeeze(adj,-1)
        # print(h.dtype)
        # print(h.shape)
        h = h.type_as(self.W)
        Wh = torch.matmul(h, self.W)  # (N, out_features)
        #print(h)
        #print(self.W)

        Wh1 = torch.matmul(Wh, self.a[:self.out_features, :])  # (N, 1)
        Wh2 = torch.matmul(Wh, self.a[self.out_features:, :])  # (N, 1)
        # print(Wh1.shape)
        # print(Wh2.shape)

        # Wh1 + Wh2.T 是N*N矩阵，第i行第j列是Wh1[i]+Wh2[j]
        # 那么Wh1 + Wh2.T的第i行第j列刚好就是文中的a^T*[Whi||Whj]
        # 代表着节点i对节点j的attention
        # print(torch.transpose(Wh2,2,1).shape)
        e = self.leakyrelu(Wh1 +torch.transpose(Wh2,2,1))  # (N, N)
        padding = (-2 ** 31) * torch.ones_like(e)  # (N, N)
        # print(adj.shape)
        # print(padding.shape)
        attention = torch.where(adj > 0, e, padding)  # (N, N)
        attention = F.softmax(attention, dim=1)  # (N, N)
        # attention矩阵第i行第j列代表node_i对node_j的注意力
        # 对注意力权重也做dropout（如果经过mask之后，attention矩阵也许是高度稀疏的，这样做还有必要吗？）
        attention = F.dropout(attention, self.dropout, training=self.training)

        h_prime = torch.matmul(attention, Wh)  # (N, out_features)
        if self.concat:
            return F.elu(h_prime)
        else:
            return h_prime
        

In [10]:
class GAT(nn.Module):
    def __init__(self,date_emb, nfeat, nhid, dropout, alpha, nheads):
        super(GAT, self).__init__()
        date_index_number,date_dim = date_emb[0], date_emb[1]
        self.dropout = dropout
        self.MH = nn.ModuleList([
            GraphAttentionLayer(nfeat, nhid, dropout, alpha, concat=True)
            for _ in range(nheads)
        ])
        self.out_att = GraphAttentionLayer(nhid * nheads, nhid, dropout, alpha, concat=False)
        self.date_embdding = nn.Embedding(date_index_number,date_dim)
        self.active_index = nn.Linear(nhid,1)
        self.consume_index = nn.Linear(nhid,1)
    def forward(self,x_date,x_feature,x_mask_data):


        x = x_feature
        # x = F.dropout(x_feature, self.dropout, training=self.training)  # (N, nfeat)
        x = torch.cat([head(x, x_mask_data) for head in self.MH], dim=-1)  # (N, nheads*nhid)
        x = F.dropout(x, self.dropout, training=self.training)  # (N, nfeat)


        # x = F.dropout(x, self.dropout, training=self.training)  # (N, nheads*nhid)
        x = self.out_att(x, x_mask_data)
        # print(x.shape,x.dtype)
        act_pre= self.active_index(x)
        con_pre = self.consume_index(x)
        return  act_pre,con_pre


In [11]:
class BILSTM(nn.Module):
    def __init__(self,date_emb, nfeat, nhid, dropout, alpha, nheads):
        super(BILSTM, self).__init__()
        date_index_number,date_dim = date_emb[0], date_emb[1]
        self.dropout = dropout
        self.lstm = nn.LSTM(nfeat,
                nhid,
                num_layers=2,
                bias=True,
                batch_first=False,
                dropout=0,
                bidirectional=True)

        self.active_index = nn.Linear(2*nhid, 1)
        self.consume_index = nn.Linear(2*nhid,1)
        self.my_linear = nn.Linear(2*nhid,128)


    def forward(self,x_date,x_feature,x_mask_data):
        x_feature = x_feature.float()
        # print(x_feature.shape)
        lstm_out, (hidden, cell) = self.lstm(x_feature)
        x = lstm_out
        # print(x.shape)


        x = F.dropout(x, self.dropout, training=self.training)  # (N, nheads*nhid)
        act_pre= self.active_index(x)
        con_pre = self.consume_index(x)
        # print(act_pre.shape,con_pre.shape)
        # return  act_pre,con_pre
        return self.my_linear(x)

In [12]:
class IntegratedModel(nn.Module):
    def __init__(self, date_emb, nfeat, gat_hidden_dim, lstm_hidden_dim, dropout, alpha, nheads):
        super(IntegratedModel, self).__init__()
      
        self.bilstm = BILSTM(date_emb, nfeat, lstm_hidden_dim, dropout, alpha, nheads)
        
        self.gat = GAT(date_emb, 128, gat_hidden_dim, dropout, alpha, nheads)
      
        self.dropout = dropout

    def forward(self, x_date,x_feature,x_mask_data):
    
        x_feature = x_feature.float()    
    
        #BILSTM处理
        lstm_ouput = self.bilstm(x_date,x_feature,x_mask_data)
        
        # GAT 处 理
        act_pre, con_pre = self.gat(x_date,lstm_ouput,x_mask_data)

        return act_pre, con_pre

In [14]:
import torch


In [13]:
def get_train_data(file_path,edge_pth):
    df = pd.read_csv(file_path, encoding='utf-8')
    #读取图文件，并且把它存储为一个名字为df的对象，并且指定文件字符编码为UFT-8编码
    edge_df = pd.read_csv(edge_pth, encoding='utf-8')
    df.head()

    # %%
    # 先定义两个空字典变量，分别用来存储地点和日期和某个序号的映射
    geohasd_df_dict = {}
    date_df_dict = {}
    #最开始字典为空，先声明两个初始值为0的变量
    number_hash = 0
    number_date = 0
    
    # 使用循环
    for i in df["geohash_id"]:

        if i not in geohasd_df_dict.keys():
            geohasd_df_dict[i] = number_hash
            number_hash += 1 #字典已经存入了一个地点，那么number_hash就应该加1
            
    for i in df["date_id"]:
        if i not in date_df_dict.keys():
            date_df_dict[i] = number_date
            number_date += 1

    # 这里创建了一个列表，行数就是日期的数目，列数是地点的数目
    # 子列表数目就是
    new_data = np.zeros((len(date_df_dict),len(geohasd_df_dict),38))
    # [len(geohasd_df_dict) * [0]] * len(date_df_dict)
    
    
    
    for index, row in df.iterrows():
    # iterrows()方法会返回一个元组，其中包含当前行的索引和数据
    # index变量将保存当前行的索引，row变量将保存当前行的数据
        # print(index)
        hash_index, date_index = geohasd_df_dict[row["geohash_id"]], date_df_dict[row["date_id"]]
        #将时间index加到里面
        """
        这里new_data[date_index][hash_index]将被赋值为一个列表。
        这个列表的第一个元素是date_index，后面是row.iloc[2:]的值。
        row.iloc[2:]表示从row中的第三个元素开始到最后一个元素的切片。
        包含一行中从第三列开始的所有列的值。
        """
        new_data[date_index][hash_index] = [date_index]+list(row.iloc[2:])
    new_data = np.array(new_data) 
    # 这里new_data转换为numpy数组，这个函数会创建一个具有相同维度和元素的新数组
    
    
    
    
    # x_train,y_train = new_data[:, :-2], new_data[:, -2:]
    # print(len(geohasd_df_dict))
    # exit()
    # print(x_train.shape)
    # print(y_train.shape)
    #这里构建邻接矩阵其中mask表示1为有边，0无边， value_mask表示有值
    #并且这里我考虑mask是一个无向图，如果有向删除x_mask[date_index][point2_index][point1_index],value_mask同理
    
    #下面两个是元素值均为0的数组
    x_mask =  np.zeros((len(date_df_dict),len(geohasd_df_dict),len(geohasd_df_dict),1), dtype = float)
    x_edge_df =np.zeros((len(date_df_dict),len(geohasd_df_dict),len(geohasd_df_dict),2), dtype = float)

    for index, row in edge_df.iterrows():
        # print(index)
        # 地点编号在字典中找不到，就说明这个数据是错误的，出现了错误的地点，那就进入下一层循环
        if row["geohash6_point1"] not in geohasd_df_dict.keys() or row["geohash6_point2"] not in geohasd_df_dict.keys():
            continue
        point1_index,point2_index,F_1,F_2,date_index= geohasd_df_dict[row["geohash6_point1"]],geohasd_df_dict[row["geohash6_point2"]]\
            ,row["F_1"],row["F_2"],date_df_dict[row["date_id"]]
        x_mask[date_index][point1_index][point2_index] = 1
        x_mask[date_index][point2_index][point1_index] = 1
        # 这里把mask数组对应位置赋值为1，说明两个地点之间有边
        x_edge_df[date_index][point1_index][point2_index] = [F_1,F_2]
        x_edge_df[date_index][point2_index][point1_index] = [F_1, F_2]
        # 把边上面的两个特征存入数组x_edge_df中
    # print(data)

    return geohasd_df_dict, date_df_dict, new_data,x_mask, x_edge_df
#get_train_data函数运行完之后，把地点、时间字典等各种数据做一个返回
#newdata存的是某个日期某个地点的具体特征
#x_mask 存的是某个日期两个地点之间是否有边
#x_edge_df存的是边上的两个特征

In [14]:
def get_test_data(file_path,edge_pth):
    df = pd.read_csv(file_path, encoding='utf-8')
    #读取图文件，并且把它存储为一个名字为df的对象，并且指定文件字符编码为UFT-8编码
    edge_df = pd.read_csv(edge_pth, encoding='utf-8')
    df.head()

    # %%
    # 先定义两个空字典变量，分别用来存储地点和日期和某个序号的映射
    geohasd_df_dict = {}
    date_df_dict = {}
    #最开始字典为空，先声明两个初始值为0的变量
    number_hash = 0
    number_date = 0
    
    # 使用循环
    for i in df["geohash_id"]:

        if i not in geohasd_df_dict.keys():
            geohasd_df_dict[i] = number_hash
            number_hash += 1 #字典已经存入了一个地点，那么number_hash就应该加1
            
    for i in df["date_id"]:
        if i not in date_df_dict.keys():
            date_df_dict[i] = number_date
            number_date += 1

    # 这里创建了一个列表，行数就是日期的数目，列数是地点的数目
    # 子列表数目就是
    new_data = np.zeros((len(date_df_dict),len(geohasd_df_dict),36))
    # [len(geohasd_df_dict) * [0]] * len(date_df_dict)
    
    
    
    for index, row in df.iterrows():
    # iterrows()方法会返回一个元组，其中包含当前行的索引和数据
    # index变量将保存当前行的索引，row变量将保存当前行的数据
        # print(index)
        hash_index, date_index = geohasd_df_dict[row["geohash_id"]], date_df_dict[row["date_id"]]
        #将时间index加到里面
        """
        这里new_data[date_index][hash_index]将被赋值为一个列表。
        这个列表的第一个元素是date_index，后面是row.iloc[2:]的值。
        row.iloc[2:]表示从row中的第三个元素开始到最后一个元素的切片。
        包含一行中从第三列开始的所有列的值。
        """
        new_data[date_index][hash_index] = [date_index]+list(row.iloc[2:])
    new_data = np.array(new_data) 
    # 这里new_data转换为numpy数组，这个函数会创建一个具有相同维度和元素的新数组
    
    
    
    
    # x_train,y_train = new_data[:, :-2], new_data[:, -2:]
    # print(len(geohasd_df_dict))
    # exit()
    # print(x_train.shape)
    # print(y_train.shape)
    #这里构建邻接矩阵其中mask表示1为有边，0无边， value_mask表示有值
    #并且这里我考虑mask是一个无向图，如果有向删除x_mask[date_index][point2_index][point1_index],value_mask同理
    
    #下面两个是元素值均为0的数组
    x_mask =  np.zeros((len(date_df_dict),len(geohasd_df_dict),len(geohasd_df_dict),1), dtype = float)
    x_edge_df =np.zeros((len(date_df_dict),len(geohasd_df_dict),len(geohasd_df_dict),2), dtype = float)

    for index, row in edge_df.iterrows():
        # print(index)
        # 地点编号在字典中找不到，就说明这个数据是错误的，出现了错误的地点，那就进入下一层循环
        if row["geohash6_point1"] not in geohasd_df_dict.keys() or row["geohash6_point2"] not in geohasd_df_dict.keys():
            continue
        point1_index,point2_index,F_1,F_2,date_index= geohasd_df_dict[row["geohash6_point1"]],geohasd_df_dict[row["geohash6_point2"]]\
            ,row["F_1"],row["F_2"],date_df_dict[row["date_id"]]
        x_mask[date_index][point1_index][point2_index] = 1
        x_mask[date_index][point2_index][point1_index] = 1
        # 这里把mask数组对应位置赋值为1，说明两个地点之间有边
        x_edge_df[date_index][point1_index][point2_index] =  [F_1,F_2]
        x_edge_df[date_index][point2_index][point1_index] = [F_1, F_2]
        # 把边上面的两个特征存入数组x_edge_df中
    # print(data)

    return     geohasd_df_dict, date_df_dict, new_data,x_mask, x_edge_df
#get_train_data函数运行完之后，把地点、时间字典等各种数据做一个返回
#newdata存的是某个日期某个地点的具体特征
#x_mask 存的是某个日期两个地点之间是否有边
#x_edge_df存的是边上的两个特征

In [15]:
# 这是一个评估函数，评估模型在给定数据集上的性能
# model是要评估的模型
# dataset是要用于评估模型的数据的数据集对象
# args是包含其他辅助参数的对象
def eval(model, dataset, args,indx):
    global min_loss
    model.eval()
    with torch.no_grad():

        dev_loss = 0.0
        for j in trange(dataset.batch_count):
            x_date, x_feature, x_mask_data, x_edge_data, x_tags = dataset.get_batch(j)
            act_pre, con_pre = model(x_date, x_feature, x_mask_data)
            predict = torch.cat((act_pre, con_pre), dim=-1)
            loss = criterion(predict, x_tags)
            dev_loss+= loss
        print("epoch {0} dev loss is {1}".format(indx,dev_loss))
        if  dev_loss < min_loss:
            min_loss = dev_loss
            # best_model_params = model.state_dict()
            # torch.save(model, 'best_model_{}.pth'.format(dev_loss))
        if(indx==args.epochs-1):
            torch.save(model, 'best_model_{}.pth'.format(dev_loss))
            torch.save(model, 'best_model_{}.pth'.format(min_loss))
            print(dev_loss)
            print(min_loss)
        # print(min_loss)
        # torch.save(model, 'best_model_{}.pth'.format(min_loss))
        model.train()

In [16]:
def train(args):

    geohasd_df_dict, date_df_dict, x_train, x_mask, x_edge_df = get_train_data('./train_90.csv',
                                                                                        "./edge_90.csv")
   
    x_train,x_dev = torch.tensor(x_train[:int(len(x_train)*args.rat)]),torch.tensor(x_train[int(len(x_train)*args.rat):])
  
   
    x_mask_train,x_mask_dev = torch.tensor(x_mask[:int(len(x_mask)*args.rat)]),torch.tensor(x_mask[int(len(x_mask)*args.rat):])
   
    x_edge_train, x_edge_dev = torch.tensor(x_edge_df[:int(len(x_edge_df) * args.rat)]),torch.tensor( x_edge_df[int(len(x_edge_df) * args.rat):])



    date_emb = 5  
    # model = BILSTM(date_emb =[len(date_df_dict),date_emb], nfeat=35, nhid=64, dropout=0.3, alpha=0.3, nheads=8).to(args.device)
    
    model = IntegratedModel(date_emb =[len(date_df_dict),date_emb], nfeat=35, gat_hidden_dim=64, lstm_hidden_dim=64,dropout=0.3, alpha=0.3, nheads=8)
  
    optimizer = torch.optim.Adam(params=model.parameters(),lr=args.lr)

    model.train()
    trainset = DataIterator(x_train,x_mask_train,x_edge_train, args)
    valset =DataIterator(x_dev,x_mask_dev,x_edge_dev, args)

 
    for indx in range(args.epochs):
        train_all_loss = 0.0
        for j in trange(trainset.batch_count):
            x_date,x_feature,x_mask_data,x_edge_data,x_tags= trainset.get_batch(j)
            act_pre, con_pre = model(x_date,x_feature,x_mask_data)
            # act_pre,con_pre = model()
            predict = torch.cat((act_pre, con_pre), dim=-1)
            loss = criterion(predict, x_tags)
            train_all_loss += loss
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        print( 'epoch {0} train loss :{1}'.format(indx,train_all_loss))
     
        eval(model,valset, args,indx)

In [18]:
# min_loss = float('inf')
# 定义一个类似 argparse 的命名元组
from collections import namedtuple
Args = namedtuple('Args', ['epochs', 'batch_size', 'device', 'lr', 'rat', 'decline'])

# 创建一个 Args 对象并设置各个参数的值
args = Args(100, 4, 'cpu', 0.0005, 0.9, 30)

train(args)

100%|██████████| 21/21 [00:24<00:00,  1.16s/it]


epoch 0 train loss :98630.78125


100%|██████████| 3/3 [00:01<00:00,  1.86it/s]


epoch 0 dev loss is 13061.85546875


100%|██████████| 21/21 [00:23<00:00,  1.10s/it]


...

epoch 98 dev loss is 173.30386352539062


100%|██████████| 21/21 [00:20<00:00,  1.01it/s]


epoch 99 train loss :4311.77001953125


100%|██████████| 3/3 [00:01<00:00,  2.20it/s]


epoch 99 dev loss is 174.20700073242188
tensor(174.2070)
tensor(169.2388)


In [ ]:
# min_loss = float('inf')
# 定义一个类似 argparse 的命名元组
from collections import namedtuple
Args = namedtuple('Args', ['epochs', 'batch_size', 'device', 'lr', 'rat', 'decline'])

# 创建一个 Args 对象并设置各个参数的值
args = Args(150, 4, 'cpu', 0.0008, 0.9, 30)

train(args)

100%|██████████████████████████████████████████████████████████████████████████████████| 21/21 [00:29<00:00,  1.41s/it]


epoch 0 train loss :93025.328125


100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:02<00:00,  1.45it/s]


epoch 0 dev loss is 10342.955078125


100%|██████████████████████████████████████████████████████████████████████████████████| 21/21 [00:29<00:00,  1.42s/it]


...

100%|██████████████████████████████████████████████████████████████████████████████████| 21/21 [00:26<00:00,  1.24s/it]


epoch 128 train loss :4199.3349609375


100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:01<00:00,  1.78it/s]


epoch 128 dev loss is 208.77081298828125



  0%|                                                                                           | 0/21 [00:00<?, ?it/s]

In [23]:
# min_loss = float('inf')
# 变换维度128（还没跑）
# 定义一个类似 argparse 的命名元组
from collections import namedtuple
Args = namedtuple('Args', ['epochs', 'batch_size', 'device', 'lr', 'rat', 'decline'])

# 创建一个 Args 对象并设置各个参数的值
args = Args(50, 8, 'cpu', 0.0009, 0.9, 30)

train(args)

100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [00:31<00:00,  2.91s/it]


epoch 0 train loss :51200.58984375


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.01it/s]


epoch 0 dev loss is 8342.8408203125


100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [00:29<00:00,  2.65s/it]


...

epoch 48 dev loss is 156.01931762695312


100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [00:29<00:00,  2.72s/it]


epoch 49 train loss :2316.36083984375


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.01it/s]


epoch 49 dev loss is 158.46142578125
tensor(158.4614)
tensor(147.9580)


In [20]:
class DataIterator2(object):
    def __init__(self, x_data,x_mask_data,x_edge_data, args):
        self.x_data,self.x_mask_data,self.x_edge_data,=x_data,x_mask_data,x_edge_data,
        #date跟fearture的分开
        self.x_date,self.x_feature=self.x_data[:,:,0],self.x_data[:,:,1:]
        # print(self.x_date.shape,self.x_feature.shape,self.x_tags.shape)
        self.args = args
        self.batch_count = math.ceil(len(x_data)/args.batch_size)

    def get_batch(self, index):
        x_date = []
        x_feature = []
        x_mask_data=[]
        x_edge_data = []

        for i in range(index * self.args.batch_size,
                       min((index + 1) * self.args.batch_size, len(self.x_data))):

            x_date.append(self.x_date[i])
            x_feature.append(self.x_feature[i].float() )

            # print(self.x_mask_data[i].shape)
            x_mask_data.append(self.x_mask_data[i])
            # print(self.x_edge_data[i].shape)
            x_edge_data.append(self.x_edge_data[i])

        x_date = torch.stack(x_date).to(self.args.device)
        x_feature = torch.DoubledTensor(torch.stack(x_feature)).to(self.args.device)
        x_mask_data = torch.stack(x_mask_data).to(self.args.device)
        x_edge_data = torch.stack(x_edge_data).to(self.args.device)


        return  x_date,x_feature,x_mask_data,x_edge_data


In [21]:
node_csv = pd.read_csv('A/node_test_4_A.csv')
edge_csv = pd.read_csv('A/edge_test_4_A.csv')
geohash_id = node_csv['geohash_id']
date_id = node_csv['date_id']
geohasd_df_dict2, date_df_dict2, x_data2, x_mask2, x_edge_df2 = get_test_data('A/node_test_4_A.csv','A/edge_test_4_A.csv')#得到测试集数据
x_data2 = torch.tensor(x_data2[:])
x_mask2 = torch.tensor(x_mask2[:])
x_edge_df2 = torch.tensor(x_edge_df2[:])
outset=DataIterator2 (x_data2,x_mask2,x_edge_df2, args)
predict = torch.Tensor()
model1 = torch.load('best_model_169.23878479003906.pth')
with torch.no_grad():
    act_pre, con_pre = model1(outset.x_date,outset.x_feature,outset.x_mask_data)
    print(act_pre.shape)
    print(con_pre.shape)
    act_pre=act_pre.reshape(1,4560)
    con_pre=con_pre.reshape(1,4560)
    predict = torch.cat((act_pre, con_pre), dim= 0)
        

consumption_level = predict[1,:].to('cpu')
activity_level = predict[0, :].to('cpu')
print(consumption_level.shape)
print(activity_level.shape)

output = {
    'geohash_id': geohash_id.tolist(),
    'consumption_level': consumption_level.tolist(),
    'activity_level': activity_level.tolist(),
    'date_id': date_id.tolist()
}
df = pd.DataFrame.from_dict(output)
df.to_csv('submit_11.21(先lstm再gat).csv', sep='\t', index=False)

torch.Size([4, 1140, 1])
torch.Size([4, 1140, 1])
torch.Size([4560])
torch.Size([4560])


In [ ]:
# import torch
# model1 = torch.load('best_model.pth')

In [ ]:
# model = GAT(date_emb =[124,6], nfeat=35, nhid=64, dropout=0.3, alpha=0.3, nheads=8).to('cpu')


In [ ]:
# torch.save(model.state_dict(),'my_model.pth')

In [ ]:
# m = torch.load('my_model.pth')

In [ ]:
# kk = model.load_state_dict(m)

In [ ]:
# kk

In [ ]:
# """
# # 定义一个类似 argparse 的命名元组
# from collections import namedtuple
#
#
# Args = namedtuple('Args', ['epochs', 'batch_size', 'device', 'lr', 'rat', 'decline'])
#
# # 创建一个 Args 对象并设置各个参数的值
# args = Args(300, 4, 'cuda', 1e-3, 0.9, 30)
# """
# geohasd_df_dict, date_df_dict, x_train, x_mask, x_edge_df = get_train_data('./train_90.csv',"./edge_90.csv")
# #分割各种训练集测试集
# x_train,x_dev = torch.tensor(x_train[:int(len(x_train)*args.rat)]),torch.tensor(x_train[int(len(x_train)*args.rat):])
# x_mask_train,x_mask_dev = torch.tensor(x_mask[:int(len(x_mask)*args.rat)]),torch.tensor(x_mask[int(len(x_mask)*args.rat):])
# x_edge_train, x_edge_dev = torch.tensor(x_edge_df[:int(len(x_edge_df) * args.rat)]),torch.tensor( x_edge_df[int(len(x_edge_df) * args.rat):])
#
# date_emb = 5
#
#
#     # 这里的x包含了date_id+F35个特征+2个y值的
# # train_activate = torch.tensor(y_train[:, -2])
# # train_consume = torch.tensor(y_train[:, -1])
#
# model = GAT(date_emb =[len(date_df_dict),date_emb], nfeat=35, nhid=64, dropout=0.3, alpha=0.3, nheads=8)
# def train(args,model):
#
#     model = model.to(args.device)
#     # rmse_loss = torch.sqrt(mse_loss)
#
#     # model = my_model.BILSTM(date_emb =[len(date_df_dict),date_emb], nfeat=35, nhid=64, dropout=0.3, alpha=0.3, nheads=8).to(args.device)
#     optimizer = torch.optim.Adam(params=model.parameters(),lr=args.lr)
#     # scheduler = torch.optim.lr_scheduler.StepLR(optimizer, args.decline, gamma=0.5, last_epoch=-1)
#     model.train()
#     trainset = DataIterator(x_train,x_mask_train,x_edge_train, args)
#     valset =DataIterator(x_dev,x_mask_dev,x_edge_dev, args)
#     for indx in range(args.epochs):
#         train_all_loss = 0.0
#         for j in trange(trainset.batch_count):
#             x_date,x_feature,x_mask_data,x_edge_data,x_tags= trainset.get_batch(j)
#             act_pre, con_pre = model(x_date,x_feature,x_mask_data)#!!!!!
#             predict = torch.cat((act_pre, con_pre), dim=-1)
#
#             loss = criterion(predict, x_tags)
#             train_all_loss += loss
#             optimizer.zero_grad()
#             loss.backward()
#             optimizer.step()
#         print('this epoch train loss :{0}'.format(train_all_loss/trainset.batch_count))
#         # scheduler.step()
#         eval(model,valset, args)
